In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')
T3 = Transition(model, 'T3')

jobclass1 = ClosedClass(model, 'Class1', 10, P1, 0)
jobclass2 = ClosedClass(model, 'Class2', 7, P1, 0)

In [3]:
# T1 - Different modes for different job classes
# Mode1: handles Class1 - requires 2 tokens
mode1 = T1.add_mode('Mode1')
T1.set_distribution(mode1, Exp(2))
T1.set_enabling_conditions(mode1, jobclass1, P1, 2)
T1.set_firing_outcome(mode1, jobclass1, P2, 2)

# Mode2: handles Class2 - requires 1 token
mode2 = T1.add_mode('Mode2')
T1.set_distribution(mode2, Exp(3))
T1.set_enabling_conditions(mode2, jobclass2, P1, 1)
T1.set_firing_outcome(mode2, jobclass2, P2, 1)

# T2 - Handles Class1 tokens individually
mode3 = T2.add_mode('Mode3')
T2.set_distribution(mode3, Erlang(1.5, 2))
T2.set_enabling_conditions(mode3, jobclass1, P2, 1)
T2.set_firing_outcome(mode3, jobclass1, P1, 1)

# T3 - Handles Class2 tokens in batches of 4
mode4 = T3.add_mode('Mode4')
T3.set_distribution(mode4, Exp(0.5))
T3.set_enabling_conditions(mode4, jobclass2, P2, 4)
T3.set_firing_outcome(mode4, jobclass2, P1, 4)

In [4]:
# Set up routing matrix for both job classes
routingMatrix = model.init_routing_matrix()

# Class1 routing
routingMatrix.set(jobclass1, jobclass1, P1, T1, 1.0)  # P1 -> T1 (Class1)
routingMatrix.set(jobclass1, jobclass1, P2, T2, 1.0)  # P2 -> T2 (Class1)
routingMatrix.set(jobclass1, jobclass1, T1, P2, 1.0)  # T1 -> P2 (Class1)
routingMatrix.set(jobclass1, jobclass1, T2, P1, 1.0)  # T2 -> P1 (Class1)

# Class2 routing
routingMatrix.set(jobclass2, jobclass2, P1, T1, 1.0)  # P1 -> T1 (Class2)
routingMatrix.set(jobclass2, jobclass2, P2, T3, 1.0)  # P2 -> T3 (Class2)
routingMatrix.set(jobclass2, jobclass2, T1, P2, 1.0)  # T1 -> P2 (Class2)
routingMatrix.set(jobclass2, jobclass2, T3, P1, 1.0)  # T3 -> P1 (Class2)

model.link(routingMatrix)

In [5]:
# Set initial state - both classes start in P1
# P1 gets [10, 7] tokens for [Class1, Class2]
P1.set_state([10, 7])  # Using Python list for multi-class
P2.set_state([0, 0])  # No tokens in P2 for either class

In [6]:
# Solve the model
solver = JMT(model, seed=23000)
avgTable = solver.avg_table()

Station JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
     P1   Class1   0.0   0.0    0.0     0.0   0.0   0.0
     P1   Class2   0.0   0.0    0.0     0.0   0.0   0.0
     P2   Class1   0.0   0.0    0.0     0.0   0.0   0.0
     P2   Class2   0.0   0.0    0.0     0.0   0.0   0.0
